In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/ai-agricultural-yield-predictor/Crop_recommendation.csv
/kaggle/input/ai-agricultural-yield-predictor/Crop_Data.xlsx.csv
/kaggle/input/ai-agricultural-yield-predictor/crop-yield.csv
/kaggle/input/ai-agricultural-yield-predictor/agriculture_dataset.csv
/kaggle/input/ai-agricultural-yield-predictor/egypt_local_crop_prices_2023_2025.csv


In [2]:
df=pd.read_csv('/kaggle/input/ai-agricultural-yield-predictor/agriculture_dataset.csv')

In [3]:
df.head()

,Farm_ID,Crop_Type,Farm_Area(acres),Irrigation_Type,Fertilizer_Used(tons),Pesticide_Used(kg),Yield(tons),Soil_Type,Season,Water_Usage(cubic meters)
0,F001,Cotton,329.40,Sprinkler,8.14,2.21,14.44,Loamy,Kharif,76648.20
1,F002,Carrot,18.67,Manual,4.77,4.36,42.91,Peaty,Kharif,68725.54
2,F003,Sugarcane,306.03,Flood,2.91,0.56,33.44,Silty,Kharif,75538.56
3,F004,Tomato,380.21,Rain-fed,3.32,4.35,34.08,Silty,Zaid,45401.23
4,F005,Tomato,135.56,Sprinkler,8.33,4.48,43.28,Clay,Zaid,93718.69


In [4]:
#show missing value in data
print(f"Missing Value = {df.isnull().sum()}")

#show duplicated value
print(f'Duplicated = {df.duplicated().sum()}')

#the shape of data
print (f'Shape = {df.shape}')

Missing Value = Farm_ID                      0
Crop_Type                    0
Farm_Area(acres)             0
Irrigation_Type              0
Fertilizer_Used(tons)        0
Pesticide_Used(kg)           0
Yield(tons)                  0
Soil_Type                    0
Season                       0
Water_Usage(cubic meters)    0
dtype: int64
Duplicated = 0
Shape = (50, 10)


In [5]:
#describe data
df.describe()

,Farm_Area(acres),Fertilizer_Used(tons),Pesticide_Used(kg),Yield(tons),Water_Usage(cubic meters)
count,50.000000,50.000000,50.000000,50.000000,50.000000
mean,254.963800,4.905400,2.398000,27.059200,56724.295600
std,139.417782,2.732689,1.438613,13.345789,27264.992053
min,12.500000,0.500000,0.140000,3.860000,5869.750000
25%,135.710000,2.437500,0.972500,16.190000,37818.152500
50%,281.980000,5.045000,2.330000,28.970000,54097.075000
75%,368.107500,6.885000,3.417500,37.860000,82240.032500
max,483.880000,9.960000,4.990000,48.020000,94754.730000


In [6]:
#data info to know numerical and categorical values
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 10 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Farm_ID                    50 non-null     object 
 1   Crop_Type                  50 non-null     object 
 2   Farm_Area(acres)           50 non-null     float64
 3   Irrigation_Type            50 non-null     object 
 4   Fertilizer_Used(tons)      50 non-null     float64
 5   Pesticide_Used(kg)         50 non-null     float64
 6   Yield(tons)                50 non-null     float64
 7   Soil_Type                  50 non-null     object 
 8   Season                     50 non-null     object 
 9   Water_Usage(cubic meters)  50 non-null     float64
dtypes: float64(5), object(5)
memory usage: 4.0+ KB


In [7]:
#show NULL value
data_null = round(df.isna().sum() / df.shape[0] * 100, 2)
data_null.to_frame(name = 'percent NULL data (%)')

,percent NULL data (%)
Farm_ID,0.0
Crop_Type,0.0
Farm_Area(acres),0.0
Irrigation_Type,0.0
Fertilizer_Used(tons),0.0
Pesticide_Used(kg),0.0
Yield(tons),0.0
Soil_Type,0.0
Season,0.0
Water_Usage(cubic meters),0.0


In [8]:
df['Yield_per_Acre'] = df['Yield(tons)'] / df['Farm_Area(acres)']


In [9]:
# Water Efficiency
df['Water_Efficiency'] = df['Yield(tons)'] / df['Water_Usage(cubic meters)']


In [10]:
# Fertilizer Efficiency
df['Fertilizer_Efficiency'] = df['Yield(tons)'] / df['Fertilizer_Used(tons)']


In [11]:
# Pesticide Efficiency
df['Pesticide_Efficiency'] = df['Yield(tons)'] / df['Pesticide_Used(kg)']


In [12]:
# Optional: Total Input Efficiency
df['Input_Efficiency'] = df['Yield(tons)'] / (
    df['Fertilizer_Used(tons)'] + df['Pesticide_Used(kg)'] + df['Water_Usage(cubic meters)']
)

In [14]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
efficiency_features = ['Yield_per_Acre', 'Water_Efficiency', 'Fertilizer_Efficiency', 'Pesticide_Efficiency', 'Input_Efficiency']
df_farm_scaled = df.copy()
df_farm_scaled[efficiency_features] = scaler.fit_transform(df[efficiency_features])


In [15]:
df_farm_scaled['Final_Efficiency_Score'] = df_farm_scaled[efficiency_features].mean(axis=1)

In [16]:
df_farm_scaled = df_farm_scaled.sort_values('Final_Efficiency_Score', ascending=False)

In [17]:
print(df_farm_scaled[['Farm_ID', 'Crop_Type', 'Final_Efficiency_Score']].head(10))


   Farm_ID  Crop_Type  Final_Efficiency_Score
10    F011       Rice                0.440216
39    F040     Cotton                0.384457
48    F049     Barley                0.327464
5     F006  Sugarcane                0.289498
47    F048     Potato                0.275370
26    F027     Cotton                0.242885
18    F019      Maize                0.236990
41    F042  Sugarcane                0.235783
1     F002     Carrot                0.234327
28    F029      Wheat                0.212701


In [18]:
df_farm_scaled.to_csv("farm_efficiency_scores.csv", index=False)
print("✅ Farm Efficiency Scores saved to 'farm_efficiency_scores.csv'")

✅ Farm Efficiency Scores saved to 'farm_efficiency_scores.csv'
